In [98]:
train = read.csv('./Stat441_KaggleProject/train_selected_adam.csv', header=TRUE)
test = read.csv('./Stat441_KaggleProject/test_selected_adam.csv', header=TRUE)

train$essential[train$essential == 0] <- 1
test$essential[test$essential == 0] <- 3
test$essential[test$essential == 1] <- 2
test$essential[test$essential == 2] <- 1
test$essential[test$essential == 3] <- 2
train$d1 <- sapply(train$label, function(x) {if (x == 0) {x} else {1} })
 


cont_cols = c(sapply(seq(45, 57), function(x) {paste("X4", toString(x), sep='')}), 'correlation', 'protein', 'chromosome')
# binary_cols = c('X94','X95','X96','X97','X298', 'X307', 'X342')
# num_cols = c(cont_cols, binary_cols)
num_cols = c(cont_cols)
train[, !names(train) %in% num_cols] <- lapply(train[, !names(train) %in% num_cols], as.factor)
test[, !names(test) %in% num_cols] <- lapply(test[, !names(test) %in% num_cols], as.factor)
# train$binarysum <- rowSums(train[binary_cols])
# train <- train[, !names(train) %in% binary_cols]
# test$binarysum <- rowSums(test[binary_cols])
# test <- test[, !names(test) %in% binary_cols]

train_size <- floor(0.7 * nrow(train))
train_ind <- sample(seq_len(nrow(train)), size = train_size, replace = FALSE)
valid_ind <- seq_len(nrow(train))[-train_ind]
train_ind <- sample(seq_len(length(train_ind)), size = floor(3 * train_size), replace = TRUE)
train_df <- train[train_ind, ]
valid_df <- train[valid_ind, ]
train_df$label <- factor(train_df$label)

In [107]:
library(randomForest)
rf_clf_d1 <-  randomForest(d1 ~ . - protein - label, data = train_df, ntrees=5000)

In [127]:
valid_preds_d1 <- predict(rf_clf_d1, valid_df)
valid_preds_d1 <- factor(valid_preds_d1, levels=levels(valid_df$d1))
valid_l <- valid_preds_d1 == valid_df$d1
sum(valid_l) / length(valid_l)

[1] 0.9441624

In [110]:
for (i in colnames(test)){
    n1 = levels(test[, i])
    n2 = levels(train[, i])
    if (length(n1) != length(n2) || sum(n1 != n2)){
#         print(i)
#         print(levels(test[, i]))
#         print(levels(train[, i]))
        test[, i] <- factor(test[, i], levels = levels(train[, i]))
    }
}
test$label = 'a'
test_preds_d1 <- predict(rf_clf_d1, test)
test$d1 <- test_preds_d1
table(test_preds_d1) / length(test_preds_d1)

test_preds_d1
        0         1 
0.3952607 0.6047393 

In [137]:
train_df_2 = train_df[train_df$label != 0, ]
valid_df_2 = valid_df[valid_df$label != 0, ]
train_df_2$label = factor(train_df_2$label)
valid_df_2$label = factor(valid_df_2$label)
table(train_df_2$label)


  1   2   3   4   5   6   7   8   9  10  11  13 
637 142 212 145 132 129  34  27  24   1   8   2 

In [138]:
rf_clf_d2 <-  randomForest(label ~ . - protein - d1, data = train_df_2, ntrees=5000)

In [144]:
valid_preds_d2 <- predict(rf_clf_d2, valid_df_2)
valid_preds_d2 <- factor(valid_preds_d2, levels=levels(valid_df_2$label))
valid_l_2 <- valid_preds_d2 == valid_df_2$label
sum(valid_l_2) / length(valid_l_2)

[1] 0.8364486

In [154]:
test_2 = test[test$d1 == 1, ]
test_2$label = predict(rf_clf_d2, test_2)
test_1 = test[test$d1 == 0, ]
test_1$label = 0
test = rbind(test_1, test_2)
table(test$label) / length(test$label)


          0           1           2           3           4           5 
0.395260664 0.377251185 0.019905213 0.128909953 0.005687204 0.008530806 
          6           8 
0.048341232 0.016113744 

In [155]:
result <- data.frame("protein" = test$protein, "label" = test$label)
result_agg <- aggregate(result,
                by = list(result$protein),
                FUN = function(x) tail(names(sort(table(x))), 1))
result_agg <- data.frame("protein" = result_agg$protein, "label" = result_agg$label)
table(result_agg$label) / length(result_agg$label)


          0           1           2           3           4           5 
0.367454068 0.469816273 0.039370079 0.073490814 0.007874016 0.013123360 
          6           8 
0.013123360 0.015748031 

In [157]:
colnames(result_agg) <- c('Key', 'Label')
write.csv(result_agg,'./result_Mar31_2.csv', row.names = FALSE, quote=FALSE)